In [1]:
from os import getcwd, path
import sys

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

from common.utils import wordpunct_space_tokenize
from config import START_TAG, STOP_TAG

In [2]:
TRAIN_PATH = path.join(BASE_PATH, 'data/CoNLL-2003/eng.train')
print(TRAIN_PATH)

/Users/2359media/Documents/botbot-nlp/data/CoNLL-2003/eng.train


In [3]:
import io
import string

def read_conll_2003(filename, tag_idx=-1):
    all_data = []

    current_txt = []
    current_tags = []
    tagset = []

    fin = io.open(filename, 'r', encoding='utf-8', newline='\n', errors='ignore')
    for line in fin:
        line = line.strip()
        if len(line) > 0: # skip blank lines
            tmp = line.split(' ')
            if tmp[0] != '-DOCSTART-':
                current_txt.append(tmp[0])
                current_tags.append(tmp[tag_idx])
                tagset.append(tmp[tag_idx])
        else:
            if len(current_txt) > 0:
                line_txt = ''
                line_tags = []
                for idx in range(len(current_txt)):
                    tokens = wordpunct_space_tokenize(current_txt[idx])
                    if idx > 0:
                        line_txt += ' ' + current_txt[idx]
                        if current_tags[idx - 1] == current_tags[idx]:
                            line_tags.extend([current_tags[idx]] * (1 + len(tokens)))
                        else:
                            line_tags.append('O')
                            line_tags.extend([current_tags[idx]] * len(tokens))
                    else:
                        line_txt += current_txt[idx]
                        line_tags.extend([current_tags[idx]] * len(tokens))
                all_data.append((line_txt, ' '.join(line_tags)))
                current_txt = []
                current_tags = []
    fin.close()

    tagset = list(set(tagset))
    tag_to_ix = {tag: key for key, tag in enumerate(tagset)}
    tag_to_ix[START_TAG] = len(tagset)
    tag_to_ix[STOP_TAG] = len(tagset) + 1

    print(tag_to_ix)
    print('Loaded %s sentences' % len(all_data))
    
    return tag_to_ix, all_data

In [4]:
tag_to_ix, training_data = read_conll_2003(TRAIN_PATH)
result = []
for sentence, tag_seq in training_data:
    tokens_in = wordpunct_space_tokenize(sentence)
    assert len(tokens_in) == len(tag_seq.split(' '))
#     print(read_tags(tokens_in, tag_seq.split(' ')))

{'O': 0, 'I-PER': 1, 'B-MISC': 2, 'I-LOC': 3, 'I-MISC': 4, 'B-LOC': 5, 'I-ORG': 6, 'B-ORG': 7, '<START>': 8, '<STOP>': 9}
Loaded 14041 sentences


In [ ]:
from entities_recognition.bilstm.model import SequenceTaggerWrapper
from entities_recognition.bilstm.train import SequenceTaggerLearner
from common.callbacks import PrintLoggerCallback, EarlyStoppingCallback

model = SequenceTaggerWrapper({'tag_to_ix': tag_to_ix})
learner = SequenceTaggerLearner(model)

In [ ]:
learner.fit(
    training_data=training_data,
    epochs=50,
    callbacks=[
        PrintLoggerCallback(log_every=1),
        EarlyStoppingCallback()
    ]
)

In [ ]:
import torch
model.eval()
torch.save(model._model.state_dict(), 'bilstm-rnn-conll2003.bin')

In [ ]:
TEST_PATH_A = path.join(BASE_PATH, 'data/CoNLL-2003/eng.testa')
TEST_PATH_B = path.join(BASE_PATH, 'data/CoNLL-2003/eng.testb')
_, testing_data_a = read_conll_2003(TEST_PATH_A)
_, testing_data_b = read_conll_2003(TEST_PATH_B)

In [ ]:
test_data = [
    'I live in Ho Chi Minh City, nice place, though my hometown is in Hanoi. I do miss it sometimes',
    'Trump’s role in midterm elections roils Republicans',
    'Kenya bans film about 2 girls in love because it’s ‘too hopeful’',
    'G.O.P. leaders and White House aides are trying to prepare President Trump for trouble in House and Senate races.'
]
model(test_data)